In [1]:
import findspark    # findspark ajuda o Python a localizar a instalação do Spark em notebooks como o Jupyter
findspark.init()  #permite que você use o pyspark no seu ambiente mesmo que o spark tenha sido instalado separadamente.

In [2]:
import requests
import os
import zipfile
from datetime import datetime
import traceback

In [3]:
url = 'https://arquivos.receitafederal.gov.br/dados/cnpj/dados_abertos_cnpj/2025-06/Simples.zip'

In [4]:
data_hoje = datetime.now().strftime('%y-%m-%d')
pasta = r"C:\Users\ResTIC16\Documents\IBGE_PROJETO\datawarehouse_ibge\data\LND"
arquivo_zip = os.path.join(pasta, f'simples_{data_hoje}.zip')
arquivo_saida = os.path.join(pasta, f'simples_{data_hoje}.csv')

In [5]:
os.makedirs(pasta,exist_ok=True)

In [6]:
try:
    print("Iniciando download do Simples Nacional...")
    response = requests.get(url, timeout=120)  # Aumentei o timeout para 2 minutos
    response.raise_for_status()
    print(f"Download concluído. Tamanho: {len(response.content) / (1024*1024):.2f} MB")

    with open(arquivo_zip, 'wb') as f:
        f.write(response.content)
    print(f"Arquivo ZIP salvo: {arquivo_zip}")

    with zipfile.ZipFile(arquivo_zip) as zip_ref:
        print("Conteúdo do ZIP:")
        for i, name in enumerate(zip_ref.namelist()):
            print(f"  {i+1}. {name}")
        
        # Busca flexível por arquivos do Simples
        simples_files = [
            n for n in zip_ref.namelist() 
            if 'SIMPLES' in n.upper() and 
               ('CSV' in n.upper() or 'D50614' in n)  # Aceita diferentes padrões
        ]
        
        if not simples_files:
            # Tenta uma busca mais ampla se não encontrar
            simples_files = [n for n in zip_ref.namelist() if 'SIMPLES' in n.upper()]
            
        if not simples_files:
            raise ValueError("Nenhum arquivo do Simples encontrado no ZIP")
        
        csv_name = simples_files[0]
        print(f"Processando arquivo: {csv_name}")

        with zip_ref.open(csv_name) as csv_file, \
             open(arquivo_saida, 'w', encoding='utf-8') as output_csv:
            
            print("Convertendo codificação...")
            total_lines = 0
            
            while True:
                # Lê em blocos maiores para melhor desempenho
                chunk = csv_file.read(1024 * 1024)  # 1MB por vez
                if not chunk:
                    break
                
                # Decodifica todo o bloco de uma vez
                decoded_chunk = chunk.decode('latin1')
                output_csv.write(decoded_chunk)
                total_lines += decoded_chunk.count('\n')
                
                # Exibe progresso a cada 500 mil linhas
                if total_lines >= 500000:
                    print(f"Linhas processadas: {total_lines:,}")
                    total_lines = 0  # Reinicia o contador
    
    print(f"Conversão concluída. Arquivo CSV salvo em: {arquivo_saida}")
    
    # Remove o ZIP após extração
    os.remove(arquivo_zip)
    print(f"Arquivo ZIP removido: {arquivo_zip}")

except Exception as erro:
    error_details = traceback.format_exc()
    print(f"\nERRO DETALHADO:\n{error_details}")
    print(f"\nERRO: {erro}")

Iniciando download do Simples Nacional...
Download concluído. Tamanho: 249.10 MB
Arquivo ZIP salvo: C:\Users\ResTIC16\Documents\IBGE_PROJETO\datawarehouse_ibge\data\LND\simples_25-07-16.zip
Conteúdo do ZIP:
  1. F.K03200$W.SIMPLES.CSV.D50614
Processando arquivo: F.K03200$W.SIMPLES.CSV.D50614
Convertendo codificação...
Linhas processadas: 515,965
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas processadas: 515,966
Linhas p